In [1]:
from pyserini.index import IndexReader 
from pyserini.search import SimpleSearcher
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import json
import torch
import numpy as np
import codecs
from bs4 import BeautifulSoup
from tqdm import tqdm
from utils import *

config = json.loads(open("config.json", "r").read())
index_path = config["index_path"]
topics_path = config["topics_path"]
qrels_path = config["qrels_path"]
index_path
device = torch.device('cpu')

/home/marco/anaconda3/envs/IR/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
index_reader = IndexReader(index_path)
searcher = SimpleSearcher(index_path)
sample_topic = "hubble space telescope"
topics = [sample_topic]

global len_C
len_C = index_reader.stats()['total_terms']
print("Collection length =", len_C)


searcher.set_bm25(0.9, 0.4)


Collection length = 174540872


In [9]:
wiki_50 = api.load('glove-wiki-gigaword-50')
wiki_300 = api.load('glove-wiki-gigaword-300')
# google = api.load('word2vec-google-news-300')


In [11]:
expand_query(sample_topic, wiki_300, 3)

'hubble space telescope telescopes hubble telescope'

# Results

In [14]:
topics = get_topics(topics_path)
#k = nro. results per topic
#n = nro. extra words per topic
k = 25
n = 0

def make_results(model, n:int, k:int = 25):
    results = ""
    for i in tqdm(topics):
        ranking = ""
        if k > 0:
            expanded_topic = expand_query(topics[i], model, n)
            hits = searcher.search(expanded_topic, k=k)
        else:
            hits = searcher.search(topics[i])
        for r, h in enumerate(hits):
            ranking += f"{i} 0 {h.docid} {r+1} {h.score} RUN1\n"
        results += ranking
        
    f = open(f'results_{n}_{k}.txt', 'w')
    f.write(results)
    f.close()
    
make_results(wiki_300, n, k=25)

100%|██████████| 250/250 [00:02<00:00, 83.67it/s]


In [16]:
ndcg25 = 0

labels_gen = query_labels_from_file(qrels_path, 'results_3_25.txt')
r = 0
for labels in labels_gen:
    ndcg25+=NDCG(labels, 25)
    r+=1
     
ndcg25/r

0.6142347126883415

In [15]:
ndcg25 = 0

labels_gen = query_labels_from_file(qrels_path, 'results_0_25.txt')
r = 0
for labels in labels_gen:
    ndcg25+=NDCG(labels, 25)
    r+=1
    
ndcg25/r

0.7017154670166613